In [1]:
!pip install Pillow

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
import os
import tensorflow as tf
import numpy as np
from tensorflow import keras
from PIL import Image
from matplotlib import pyplot as plt


print(tf.__version__)

2.0.0


In [3]:
tf.random.set_seed(22)
np.random.seed(22)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
assert tf.__version__.startswith('2.')

In [4]:
# 作图函数
def save_images(imgs, name):
    new_im = Image.new('L', (280, 280))
    index = 0
    for i in range(0, 280, 28):
        for j in range(0, 280, 28):
            im = imgs[index]
            im = Image.fromarray(im, mode='L')
            new_im.paste(im, (i, j))
            index += 1
    new_im.save(name)

In [5]:
h_dim = 20
batchsz = 512
learning_rate = 1e-3

In [6]:
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train, x_test = x_train.astype(np.float32) / 255, x_test.astype(np.float32) / 255

In [7]:
train_db = tf.data.Dataset.from_tensor_slices(x_train)
train_db = train_db.shuffle(20).batch(batchsz)
test_db = tf.data.Dataset.from_tensor_slices(x_test)
test_db = test_db.batch(batchsz)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [8]:
from tensorflow.keras import Sequential
class AE(keras.Model):
    def __init__(self):
        super(AE, self).__init__()
        # encoders
        self.encoder = Sequential(
        [
            keras.layers.Dense(256, activation=tf.nn.relu),
            keras.layers.Dense(128, activation=tf.nn.relu),
            keras.layers.Dense(h_dim)
        ])
        
        self.decoder = Sequential(
        [
            keras.layers.Dense(128, activation=tf.nn.relu),
            keras.layers.Dense(256, activation=tf.nn.relu),
            keras.layers.Dense(784)
        ])
        
    def call(self, inputs, training=None):
        # 编码
        h = self.encoder(inputs)
        # 重建
        x_hat = self.decoder(h)
        return x_hat

In [9]:
model = AE()
model.build(input_shape=(None, 784))
model.summary()

Model: "ae"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      multiple                  236436    
_________________________________________________________________
sequential_1 (Sequential)    multiple                  237200    
Total params: 473,636
Trainable params: 473,636
Non-trainable params: 0
_________________________________________________________________


In [10]:
optimizer = keras.optimizers.Adam()

for epoch in range(30):
    for step, x in enumerate(train_db):
        # reshape
        x = tf.reshape(x, [-1, 784])
        with tf.GradientTape() as tape:
            x_rebuild_logit = model(x)
            rec_loss = tf.losses.binary_crossentropy(x, x_rebuild_logit, from_logits=True)
            rec_loss = tf.reduce_mean(rec_loss)
        grads = tape.gradient(rec_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        
        if step % 100 == 0:
            print(epoch,step, float(rec_loss))
        
        # evaluation
        x = next(iter(test_db))
        logits = model(tf.reshape(x, [-1, 784]))
        x_hat = tf.sigmoid(logits)
        # 
        x_hat = tf.reshape(x_hat, [-1, 28, 28])
#         x_concat = tf.concat([x, x_hat], axis=0)
        x_concat = x_hat
        x_concat = x_concat.numpy() * 255.
        x_concat = x_concat.astype(np.uint8)
        save_images(x_concat, 'ae_images/rec_epoch_%d.png'%epoch)

0 0 0.6927081346511841
0 100 0.33256739377975464
1 0 0.3180384337902069
1 100 0.31376397609710693
2 0 0.3057126998901367
2 100 0.30603522062301636
3 0 0.2970661222934723


KeyboardInterrupt: 